In [1]:
import openai
from pathlib import Path
import json
import llm2geneset

In [2]:
import re

def clean_elements(array):
    cleaned_array = []
    for element in array:
        # Use regular expression to remove (GO:xxx) substring,  R-HSA-xxx substrings, and WPxxx substrings
        cleaned_element = re.sub(r'\s*\(GO:\d+\)\s*|\s*R-HSA-\d+\s*|\s*WP\d+\s*', '', element)
        cleaned_array.append(cleaned_element)
    return cleaned_array

In [3]:
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", "WikiPathway_2023_Human"]
lib_names = ["KEGG_2021_Human"]

Export enrichr libraries as JSON.

In [4]:
for lib_name in lib_names:
    (descr, genes) = llm2geneset.read_gmt("libs_human/gmt/" + lib_name + ".txt")
    print(len(descr))

320


In [17]:
aclient = openai.AsyncClient()
models = ["gpt-3.5-turbo-0125"]#, "gpt-4o-2024-05-13"]
for model in models:
    for lib_name in lib_names:
        (descr, genes) = llm2geneset.read_gmt("libs_human/gmt/" + lib_name + ".txt")
        descr_cleaned = clean_elements(descr)
        llm_genes = await llm2geneset.get_genes(aclient, descr_cleaned, modelg=model)
        ai_lib_json = Path("libs_human/" + model + "/" + lib_name + '.json')
        if not ai_lib_json.is_file():
            ai_lib = dict(zip(descr_cleaned, llm_genes))
            json.dump(ai_lib, open(str(ai_lib_json), 'w'), indent=4)
            #ai_lib = await llm2geneset.get_genes(aclient, descr, modelg=model)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:30<00:00, 10.45it/s]
